In [1]:
from coincidence_matching import match_parts, get_export_id_types
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from brepmatching.data import BRepMatchingDataset

c:\Users\Ben\anaconda3\envs\brepmatching\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [65]:
import os
from pathlib import Path
str(Path.home().joinpath('.config','onshapecreds.json'))

'C:\\Users\\Ben\\.config\\onshapecreds.json'

In [58]:
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
from apikey.onshape import Onshape
from pathlib import Path
import os

api = Onshape(stack='https://cad.onshape.com', creds=str(Path.home().joinpath('.config','onshapecreds.json')), logging=False)

with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    variations[:100]
    variations = variations[variations.fail == 0]

    failures = []
    written = set()
    with ZipFile('../data/redownload.zip','w') as wzf:
        for i in tqdm(range(len(variations))):
            d = variations.iloc[i]

            response_orig = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_orig}/e/{d.eid}/parasolid', query={'includeExportIds':True})
            response_var = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})

            if response_orig.status_code != 200 or response_var.status_code != 200:
                failures.append(d)
                continue
            
            orig_path = 'data/BrepsWithReference/' + d.ps_orig
            var_path = 'data/BrepsWithReference/' + d.ps_var

            if orig_path not in written:
                with wzf.open(orig_path, 'w') as f:
                    f.write(response_orig.text.encode('utf-8'))
                written.add(orig_path)
            if var_path not in written:
                with wzf.open(var_path, 'w') as f:
                    f.write(response_var.text.encode('utf-8'))
                written.add(var_path)

  0%|          | 0/6611 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [51]:
#with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:

with ZipFile('../data/GeoV2FullRunBaseline.zip','r') as zf:

    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)

    data = variations[:100]
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]

    incomplete = []
    incompatible = []
    diff_geo = []

    m_f = []
    m_e = []
    m_v = []

    A = []
    B = []
    M = []

    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        path_a = 'data/BrepsWithReference/' + d.ps_orig
        path_b = 'data/baseline/' + d.baselineOrig

        path_c = 'data/BrepsWithReference/' + d.ps_var
        path_d = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch

        with zf.open(path_a,'r') as f:
            part_data_a = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_b = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_c = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_d = f.read().decode('utf-8')
        with zf.open(path_gt_match,'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            bl_match = json.load(f)

        p_a = Part(part_data_a)
        p_b = Part(part_data_b)
        p_c = Part(part_data_c)
        p_d = Part(part_data_d)

        parts_have_same_num = len(p_a.brep.nodes.faces) == len(p_b.brep.nodes.faces) and len(p_a.brep.nodes.edges) == len(p_b.brep.nodes.edges) and len(p_a.brep.nodes.vertices) == len(p_b.brep.nodes.vertices)

        matching = match_parts(part_data_a, part_data_b, True)
        complete_matching = len(matching.face_matches) == len(p_a.brep.nodes.faces) and len(matching.edge_matches) == len(p_a.brep.nodes.edges) and len(matching.vertex_matches) == len(p_a.brep.nodes.vertices)

        n_face_matches_missing = -len(matching.face_matches) + len(p_a.brep.nodes.faces)
        n_edge_matches_missing = -len(matching.edge_matches) + len(p_a.brep.nodes.edges)
        n_vertex_matches_missing =-len(matching.vertex_matches) + len(p_a.brep.nodes.vertices)

        m_f.append(n_face_matches_missing)
        m_e.append(n_edge_matches_missing)
        m_v.append(n_vertex_matches_missing)

        exact_geo = (p_a.mesh.V == p_b.mesh.V).all()

        if not parts_have_same_num:
            incompatible.append(i)
        if not complete_matching:
            incomplete.append(i)

        if not exact_geo:
            diff_geo.append(i)

        A.append(p_a)
        B.append(p_b)
        M.append(matching)


        # Exact matching of originals seems to work (and the tesselations are mostly identical!)
        # Let's see how well we match the versions with variations:

        var_matching = match_parts(part_data_c, part_data_d, True)

        id_classes_a = get_export_id_types(part_data_a)
        id_classes_b = get_export_id_types(part_data_b)
        id_classes_c = get_export_id_types(part_data_c)
        id_classes_d = get_export_id_types(part_data_d)

        good_classes = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']

        #gt_topo_matches = [(v['val1'],v['val1']) for k,v in gt_match.items() if id_classes_a[v['val1']] in good_classes and id_classes_c[v['val2']] in good_classes]
        bl_topo_matches = [(v['val1'],v['val1']) for k,v in bl_match.items() if id_classes_b[v['val1']] in good_classes and id_classes_d[v['val2']] in good_classes]

        


  1%|          | 1/99 [00:00<00:08, 10.99it/s]


KeyError: 'JF4'

In [45]:
'JF4' in part_data_d

False

In [48]:
data

,name,did,wid,eid,link_orig,mv_orig,ps_orig,mv_var,ps_var,matchFile,codeVersion,fail,seed,link_var,baselineOrig,baselineNew,baselineMatch,translationFail
0,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,8f868ab8ff16bbca8df8f914,1be1aa77cc97fec37f31922c,b459723517ecd2dbff382c8e,https://cad.onshape.com/documents/8f868ab8ff16...,2e43732de79d2a8bf3a1bac6,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,ce6be646700055037d4f196d,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,MCV2,0,10472,https://cad.onshape.com/documents/8f868ab8ff16...,,,,0
1,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,8f868ab8ff16bbca8df8f914,1be1aa77cc97fec37f31922c,b459723517ecd2dbff382c8e,https://cad.onshape.com/documents/8f868ab8ff16...,2e43732de79d2a8bf3a1bac6,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,adbc3addb10f415048c9fd9d,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,MCV2,0,71969,https://cad.onshape.com/documents/8f868ab8ff16...,baselineOrig_d65b13c6430b30819cb6b169_ab94f8bb...,baselineNew_d65b13c6430b30819cb6b169_ab94f8bbf...,baselineMatch_d65b13c6430b30819cb6b169_ab94f8b...,0
2,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,8f868ab8ff16bbca8df8f914,1be1aa77cc97fec37f31922c,b459723517ecd2dbff382c8e,https://cad.onshape.com/documents/8f868ab8ff16...,2e43732de79d2a8bf3a1bac6,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,e3dcff1488019e2b28e49556,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf8...,MCV2,0,71096,https://cad.onshape.com/documents/8f868ab8ff16...,baselineOrig_d65b13c6430b30819cb6b169_ab94f8bb...,baselineNew_d65b13c6430b30819cb6b169_ab94f8bbf...,baselineMatch_d65b13c6430b30819cb6b169_ab94f8b...,0
3,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,02481b82115d2823a1935332,f9fb5b74f41d8a5ed76ee025,7a343841c92f47503b5efd2a,https://cad.onshape.com/documents/02481b82115d...,39ca0303fd8b0656a873d967,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,d31bce2f1ebdc43ac1b16c76,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,MCV2,0,48061,https://cad.onshape.com/documents/02481b82115d...,baselineOrig_bd5fdae5f33d665982b3e072_8a7f56cc...,baselineNew_bd5fdae5f33d665982b3e072_8a7f56ccf...,baselineMatch_bd5fdae5f33d665982b3e072_8a7f56c...,0
4,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,02481b82115d2823a1935332,f9fb5b74f41d8a5ed76ee025,7a343841c92f47503b5efd2a,https://cad.onshape.com/documents/02481b82115d...,39ca0303fd8b0656a873d967,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,19e96245403cb74a68a3066f,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,bd5fdae5f33d665982b3e072_8a7f56ccff092e6146aa6...,MCV2,0,14417,https://cad.onshape.com/documents/02481b82115d...,baselineOrig_bd5fdae5f33d665982b3e072_8a7f56cc...,baselineNew_bd5fdae5f33d665982b3e072_8a7f56ccf...,baselineMatch_bd5fdae5f33d665982b3e072_8a7f56c...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bf78558e7828a79e8048eaab_581db1f3063bf2e7a8c1a...,e4076819850cfbd9e718cbd9,2feda2aa008e844acbdc3495,b402400af2a2ca0709ffaadc,https://cad.onshape.com/documents/e4076819850c...,b699725690c1eb06388e432b,bf78558e7828a79e8048eaab_581db1f3063bf2e7a8c1a...,d2a0bef30ece44dd539b7b67,bf78558e7828a79e8048eaab_581db1f3063bf2e7a8c1a...,bf78558e7828a79e8048eaab_581db1f3063bf2e7a8c1a...,MCV2,0,70860,https://cad.onshape.com/documents/e4076819850c...,baselineOrig_bf78558e7828a79e8048eaab_581db1f3...,baselineNew_bf78558e7828a79e8048eaab_581db1f30...,baselineMatch_bf78558e7828a79e8048eaab_581db1f...,0
96,ccfc70d2755e38329f174604_a2344d2c1b2d5eeefb6a7...,b094a55be71664f2fc88fe08,411478b6ab20c047e5c6c2b9,742b2f8609208d9c9c7c003a,https://cad.onshape.com/documents/b094a55be716...,bff49e582acb33827d1821ad,ccfc70d2755e38329f174604_a2344d2c1b2d5eeefb6a7...,e516f84dd20d61e06b839e21,ccfc70d2755e38329f174604_a2344d2c1b2d5eeefb6a7...,ccfc70d2755e38329f174604_a2344d2c1b2

In [39]:
d.link_var

'https://cad.onshape.com/documents/364a86f19fd247756ad7a9ae/w/7e7e27b4d79c3b03ee593b7b/m/889c25b4e40f410d427c5e41/e/ff0b76d7b2a3a69be144b793'

In [3]:
part_b = Part(part_data_b)

In [4]:
len([e.export_id for e in part_b.brep.nodes.edges])

25

In [5]:
len([k for k,v in id_classes_b.items() if v == 'PK_CLASS_edge'])

25

In [6]:
id_classes_b

{'IF': 'PK_CLASS_vertex',
 'JF0': 'PK_CLASS_vertex',
 'JF1': 'PK_CLASS_edge',
 'JF5': 'PK_CLASS_edge',
 'JF9': 'PK_CLASS_edge',
 'JFB': 'PK_CLASS_edge',
 'JFC': 'PK_CLASS_face',
 'JFD': 'PK_CLASS_body',
 'JFE': 'PK_CLASS_vertex',
 'JFF': 'PK_CLASS_edge',
 'JFG': 'PK_CLASS_face',
 'JFI': 'PK_CLASS_vertex',
 'JFJ': 'PK_CLASS_edge',
 'JFK': 'PK_CLASS_face',
 'JFM': 'PK_CLASS_vertex',
 'JFN': 'PK_CLASS_edge',
 'JFO': 'PK_CLASS_face',
 'JFQ': 'PK_CLASS_vertex',
 'JFR': 'PK_CLASS_edge',
 'JFS': 'PK_CLASS_face',
 'JFU': 'PK_CLASS_vertex',
 'JFV': 'PK_CLASS_edge',
 'JFW': 'PK_CLASS_face',
 'JFY': 'PK_CLASS_vertex',
 'JFZ': 'PK_CLASS_edge',
 'JFa': 'PK_CLASS_face',
 'JFc': 'PK_CLASS_vertex',
 'JFd': 'PK_CLASS_edge',
 'JFe': 'PK_CLASS_face',
 'JFg': 'PK_CLASS_vertex',
 'JFh': 'PK_CLASS_edge',
 'JFi': 'PK_CLASS_face',
 'JFk': 'PK_CLASS_vertex',
 'JFl': 'PK_CLASS_edge',
 'JFm': 'PK_CLASS_face',
 'JFo': 'PK_CLASS_vertex',
 'JFp': 'PK_CLASS_edge',
 'JFq': 'PK_CLASS_face',
 'JFs': 'PK_CLASS_vertex',


In [7]:
d.link_var

'https://cad.onshape.com/documents/364a86f19fd247756ad7a9ae/w/7e7e27b4d79c3b03ee593b7b/m/889c25b4e40f410d427c5e41/e/ff0b76d7b2a3a69be144b793'

In [8]:
data.baselineMatch

0     baselineMatch_d65b13c6430b30819cb6b169_ab94f8b...
1     baselineMatch_d65b13c6430b30819cb6b169_ab94f8b...
2     baselineMatch_d65b13c6430b30819cb6b169_ab94f8b...
3     baselineMatch_bd5fdae5f33d665982b3e072_8a7f56c...
4     baselineMatch_bd5fdae5f33d665982b3e072_8a7f56c...
                            ...                        
95    baselineMatch_bf78558e7828a79e8048eaab_581db1f...
96    baselineMatch_ccfc70d2755e38329f174604_a2344d2...
97    baselineMatch_ccfc70d2755e38329f174604_a2344d2...
98    baselineMatch_ccfc70d2755e38329f174604_a2344d2...
99    baselineMatch_59a2ed79d74ce92b76b00079_cebed17...
Name: baselineMatch, Length: 95, dtype: object

In [10]:
from apikey.onshape import Onshape

[INFO]: 01/06/23 17:10:21 - onshape instance created: url = https://cad.onshape.com, access key = b'CP6wrtLolmvpprBj4TPfSW2e'


In [28]:
response = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})#, query={'partIds':['JFD'],'includeExportIds':True})

[INFO]: 01/06/23 17:17:33 - nonce created: APU53ZQc0i6C2WTz1xxpnET5e
[INFO]: 01/06/23 17:17:33 - {'query': 'includeExportIds=True', 'hmac_str': b'get\napu53zqc0i6c2wtz1xxpnet5e\nsat, 07 jan 2023 01:17:33 gmt\napplication/json\n/api/partstudios/d/364a86f19fd247756ad7a9ae/m/889c25b4e40f410d427c5e41/e/ff0b76d7b2a3a69be144b793/parasolid\nincludeexportids=true\n', 'signature': b'x96l42hE1mr8B/POZ+lb9RwaYD3WRxqSa6lsAINVk0I=', 'auth': 'On CP6wrtLolmvpprBj4TPfSW2e:HmacSHA256:x96l42hE1mr8B/POZ+lb9RwaYD3WRxqSa6lsAINVk0I='}
[INFO]: 01/06/23 17:17:33 - {}
[INFO]: 01/06/23 17:17:33 - {'Content-Type': 'application/json', 'Date': 'Sat, 07 Jan 2023 01:17:33 GMT', 'On-Nonce': 'APU53ZQc0i6C2WTz1xxpnET5e', 'Authorization': 'On CP6wrtLolmvpprBj4TPfSW2e:HmacSHA256:x96l42hE1mr8B/POZ+lb9RwaYD3WRxqSa6lsAINVk0I=', 'User-Agent': 'Onshape Python Sample App', 'Accept': 'application/json'}
[INFO]: 01/06/23 17:17:33 - request url: https://cad.onshape.com/api/partstudios/d/364a86f19fd247756ad7a9ae/m/889c25b4e40f410d

In [30]:
response.text == part_data_b

False

In [13]:
d.mv_var

'889c25b4e40f410d427c5e41'

In [14]:
d.did

'364a86f19fd247756ad7a9ae'

In [15]:
d.eid

'ff0b76d7b2a3a69be144b793'

In [17]:
id_classes_b

{'IF': 'PK_CLASS_vertex',
 'JF0': 'PK_CLASS_vertex',
 'JF1': 'PK_CLASS_edge',
 'JF5': 'PK_CLASS_edge',
 'JF9': 'PK_CLASS_edge',
 'JFB': 'PK_CLASS_edge',
 'JFC': 'PK_CLASS_face',
 'JFD': 'PK_CLASS_body',
 'JFE': 'PK_CLASS_vertex',
 'JFF': 'PK_CLASS_edge',
 'JFG': 'PK_CLASS_face',
 'JFI': 'PK_CLASS_vertex',
 'JFJ': 'PK_CLASS_edge',
 'JFK': 'PK_CLASS_face',
 'JFM': 'PK_CLASS_vertex',
 'JFN': 'PK_CLASS_edge',
 'JFO': 'PK_CLASS_face',
 'JFQ': 'PK_CLASS_vertex',
 'JFR': 'PK_CLASS_edge',
 'JFS': 'PK_CLASS_face',
 'JFU': 'PK_CLASS_vertex',
 'JFV': 'PK_CLASS_edge',
 'JFW': 'PK_CLASS_face',
 'JFY': 'PK_CLASS_vertex',
 'JFZ': 'PK_CLASS_edge',
 'JFa': 'PK_CLASS_face',
 'JFc': 'PK_CLASS_vertex',
 'JFd': 'PK_CLASS_edge',
 'JFe': 'PK_CLASS_face',
 'JFg': 'PK_CLASS_vertex',
 'JFh': 'PK_CLASS_edge',
 'JFi': 'PK_CLASS_face',
 'JFk': 'PK_CLASS_vertex',
 'JFl': 'PK_CLASS_edge',
 'JFm': 'PK_CLASS_face',
 'JFo': 'PK_CLASS_vertex',
 'JFp': 'PK_CLASS_edge',
 'JFq': 'PK_CLASS_face',
 'JFs': 'PK_CLASS_vertex',
